In [3]:
from google.colab import files
uploaded = files.upload()  # Upload your `train_data_10k.json`, `eval_data_1k.json`, `test_data_1k.json`

Saving train_data_10k.json to train_data_10k.json


In [12]:

import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
import torch

In [4]:
from google.colab import files
uploaded = files.upload()  # Upload your `train_data_10k.json`, `eval_data_1k.json`, `test_data_1k.json`

Saving eval_data_1k.json to eval_data_1k.json


Saving eval_data_1k.json to eval_data_1k (1).json


In [5]:
from google.colab import files
uploaded = files.upload()  # Upload your `train_data_10k.json`, `eval_data_1k.json`, `test_data_1k.json`

Saving test_data_1k.json to test_data_1k.json


In [6]:
import json
from datasets import Dataset

def load_json_lines(path):
    with open(path, "r") as f:
        return [json.loads(line) for line in f]

train_data = load_json_lines("train_data_10k.json")
eval_data = load_json_lines("eval_data_1k.json")
test_data = load_json_lines("test_data_1k.json")

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
test_dataset = Dataset.from_list(test_data)

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "google/byt5-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
MAX_INPUT_LENGTH = 256
MAX_TARGET_LENGTH = 256

def preprocess_function(examples):
    # Prefix is optional but useful to specify task
    inputs = [f"convert pfd to pid: {pfd}" for pfd in examples["PFD"]]
    targets = examples["PID"]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length",
        return_tensors=None,
    )

    # Tokenize targets with target tokenizer context (important for some tokenizers)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding="max_length",
            return_tensors=None,
        )

    # Replace padding token id's of the labels by -100 so they are ignored by the loss
    model_inputs["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in labels["input_ids"]
    ]

    return model_inputs

# Tokenize datasets
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_dataset.column_names)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(384, 1472)
  (encoder): T5Stack(
    (embed_tokens): Embedding(384, 1472)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1472, out_features=384, bias=False)
              (k): Linear(in_features=1472, out_features=384, bias=False)
              (v): Linear(in_features=1472, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=1472, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1472, out_features=3584, bias=False)
              (wi_1): Linear(in_features=1472, out_features=3584, bias=False)
              (w

In [16]:
import torch
from transformers import TrainingArguments, Trainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir="./byt5_pid_model",
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    save_total_limit=20,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="tensorboard",
    fp16=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

Step,Training Loss,Validation Loss
100,0.000000,nan


KeyboardInterrupt: 

In [11]:
model_inputs["labels"] = [
    [(label if label != tokenizer.pad_token_id else -100) for label in labels_seq]
    for labels_seq in labels["input_ids"]
]

NameError: name 'labels' is not defined

In [ ]:
tokenizer.save_pretrained("./byt5_pid_model")
model.save_pretrained("./byt5_pid_model")

In [ ]:
def pfd_to_pid(pfd_text):
    input_text = f"convert pfd to pid: {pfd_text}"
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=MAX_INPUT_LENGTH, truncation=True).to(device)

    outputs = model.generate(
        inputs,
        max_length=2000,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
import pandas as pd

df_test = pd.read_json("test_data_1k.json", lines=True)

sample_index = 3
sample_pfd = df_test.loc[sample_index, "PFD"]
print("\n=== Inference Test ===")
print("Input PFD:", sample_pfd)
print("Predicted PID:", pfd_to_pid(sample_pfd))
print("True PID:", df_test.loc[sample_index, "PID"])

In [7]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq


In [22]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# File paths (adjust paths if needed)
train_file = "train_data_10k.json"
eval_file = "eval_data_1k.json"
test_file = "test_data_1k.json"

# Load datasets (assuming JSON Lines or JSON arrays with 'input' and 'output' keys)
data_files = {
    "train": train_file,
    "validation": eval_file,
    "test": test_file
}

dataset = load_dataset("json", data_files=data_files)

print(f"Dataset loaded: {dataset}")

model_name = "google/byt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

max_length = 512  # adjust based on your data

def preprocess_function(examples):
    inputs = examples["input"]
    targets = examples["output"]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

    # Replace pad token id's in labels by -100 to ignore in loss
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_seq]
        for label_seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess all splits
tokenized_datasets = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./byt5_pid_model",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=20,
    report_to="none",
    seed=42,
    fp16=True,  # enable if your GPU supports it
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Evaluate on validation set
eval_results = trainer.evaluate()
print("Validation results:", eval_results)

# Optionally, generate predictions on test set
print("\nSample predictions on test set:")
for i in range(3):  # print 3 samples
    input_text = dataset["test"][i]["input"]
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    outputs = model.generate(**inputs, num_beams=5, max_length=max_length)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nInput:\n{input_text}\nPrediction:\n{pred}\nExpected Output:\n{dataset['test'][i]['output']}")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [1]:
from google.colab import files
import json
from datasets import Dataset

# Upload files
uploaded = files.upload()
uploaded = files.upload()
uploaded = files.upload()

def load_jsonl(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Replace with your actual uploaded filenames
train_data = load_jsonl('train_data_10k.json')
eval_data = load_jsonl('eval_data_1k.json')
test_data = load_jsonl('test_data_1k.json')

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
test_dataset = Dataset.from_list(test_data)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

print("Sample train example:", train_dataset[0])

Saving eval_data_1k.json to eval_data_1k.json


Saving test_data_1k.json to test_data_1k.json


Saving train_data_10k.json to train_data_10k.json
Train dataset size: 10000
Eval dataset size: 1000
Test dataset size: 1000
Sample train example: {'PFD': '(raw)(hex){1}(hex){2}(mix)<2(r)[{tout}(v)(prod)]{bout}(v)(splt)[(hex){2}(hex){3}(pp)(v)(mix)<1(r)[{bout}(v)(prod)]{tout}(v)(splt)[(hex){4}(r)[{tout}(v)(prod)]{bout}(v)(hex){4}(prod)](v)1](v)2n|(raw)(hex){1}(v)(prod)n|(raw)(hex){3}(v)(prod)', 'PID': '(raw)(hex){1}(C){TC}_1(hex){2}(mix)<2(r)<_2[(C){TC}_2][(C){LC}_3][{tout}(C){PC}_4(v)<_4(prod)]{bout}(v)<_3(splt)[(hex){2}(hex){3}(C){TC}_5(pp)[(C){M}](C){PI}(C){FC}_6(v)<_6(mix)<1(r)<_7[(C){TC}_7][(C){LC}_8][{bout}(v)<_8(prod)]{tout}(C){PC}_9(v)<_9(splt)[(C){FC}_10(v)1<_10](hex){4}(r)<_11[(C){TC}_11][(C){LC}_12][{tout}(C){PC}_13(v)<_13(prod)]{bout}(v)<_12(hex){4}(prod)](C){FC}_14(v)2<_14n|(raw)(hex){1}(v)<_1(prod)n|(raw)(hex){3}(v)<_5(prod)'}


In [2]:


from transformers import ByT5Tokenizer

tokenizer = ByT5Tokenizer.from_pretrained('google/byt5-small')
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['PFD']
    targets = examples['PID']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

    # Replace padding token id's in labels by -100 to ignore them in loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import torch
model = AutoModelForSeq2SeqLM.from_pretrained('google/byt5-small')

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
train_dataset = train_dataset.filter(lambda x: x['PFD'] is not None and x['PID'] is not None and len(x['PFD']) > 0 and len(x['PID']) > 0)

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
print (train_dataset)

Dataset({
    features: ['PFD', 'PID', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})


In [13]:
def preprocess_function(examples):
    inputs = examples['PFD']
    targets = examples['PID']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

    labels_ids = labels["input_ids"]
    # Replace padding token id's of the labels by -100 so it's ignored by the loss
    labels_ids = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels_ids
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

batch = train_dataset[2]
input_ids = torch.tensor(batch['input_ids']).unsqueeze(0).to(device)
labels = torch.tensor(batch['labels']).unsqueeze(0).to(device)

outputs = model(input_ids=input_ids, labels=labels)
print("Loss:", outputs.loss.item())

Loss: nan


In [4]:
training_args = TrainingArguments(
    output_dir='./byt5_finetuned',
    eval_strategy='steps',  # also note: eval_strategy -> evaluation_strategy
    eval_steps=100,
    logging_steps=200,
    save_steps=100,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
)

In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

<ipython-input-5-c6c4fdb81e77>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [45]:
import transformers
print(transformers.__version__)

4.52.3


In [6]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dhritichandan1 (dhritichandan1-9) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,No log,nan
200,0.000000,nan
300,0.000000,nan
400,0.000000,nan


KeyboardInterrupt: 

In [ ]:
metrics = trainer.evaluate()
print(metrics)
trainer.save_model('./byt5_finetuned')
tokenizer.save_pretrained('./byt5_finetuned')

In [ ]:
predictions = trainer.predict(test_dataset)
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Print some example predictions
for pred in decoded_preds[:5]:
    print(pred)

In [ ]:
inputs = tokenizer(["your input text here"], return_tensors="pt", padding=True, truncation=True)

outputs = model.generate(**inputs, max_length=512)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [32]:
from transformers import ByT5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
from datasets import load_dataset

# 1. Load tokenizer and model
tokenizer = ByT5Tokenizer.from_pretrained("google/byt5-small")
model = T5ForConditionalGeneration.from_pretrained("google/byt5-small")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Load your dataset - replace with your dataset loading method

def load_jsonl(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Replace with your actual uploaded filenames
train_data = load_jsonl('train_data_10k.json')
eval_data = load_jsonl('eval_data_1k.json')
test_data = load_jsonl('test_data_1k.json')

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)
test_dataset = Dataset.from_list(test_data)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

print("Sample train example:", train_dataset[0])

# 3. Filter out bad data (empty or None)
def filter_func(example):
    return (example['PFD'] is not None and example['PID'] is not None
            and len(example['PFD'].strip()) > 0 and len(example['PID'].strip()) > 0)

train_dataset = train_dataset.filter(filter_func)
eval_dataset = eval_dataset.filter(filter_func)

# 4. Preprocessing function with padding and labels masked by -100
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['PFD']
    targets = examples['PID']

    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding='max_length'
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=max_target_length, truncation=True, padding='max_length'
        )

    # Replace pad token id's in labels by -100 to ignore during loss computation
    labels_ids = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Remove original columns (optional)
train_dataset = train_dataset.remove_columns(["PFD", "PID"])
eval_dataset = eval_dataset.remove_columns(["PFD", "PID"])

# 5. Data collator (handles batching correctly)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


Train dataset size: 10000
Eval dataset size: 1000
Test dataset size: 1000
Sample train example: {'PFD': '(raw)(hex){1}(hex){2}(mix)<2(r)[{tout}(v)(prod)]{bout}(v)(splt)[(hex){2}(hex){3}(pp)(v)(mix)<1(r)[{bout}(v)(prod)]{tout}(v)(splt)[(hex){4}(r)[{tout}(v)(prod)]{bout}(v)(hex){4}(prod)](v)1](v)2n|(raw)(hex){1}(v)(prod)n|(raw)(hex){3}(v)(prod)', 'PID': '(raw)(hex){1}(C){TC}_1(hex){2}(mix)<2(r)<_2[(C){TC}_2][(C){LC}_3][{tout}(C){PC}_4(v)<_4(prod)]{bout}(v)<_3(splt)[(hex){2}(hex){3}(C){TC}_5(pp)[(C){M}](C){PI}(C){FC}_6(v)<_6(mix)<1(r)<_7[(C){TC}_7][(C){LC}_8][{bout}(v)<_8(prod)]{tout}(C){PC}_9(v)<_9(splt)[(C){FC}_10(v)1<_10](hex){4}(r)<_11[(C){TC}_11][(C){LC}_12][{tout}(C){PC}_13(v)<_13(prod)]{bout}(v)<_12(hex){4}(prod)](C){FC}_14(v)2<_14n|(raw)(hex){1}(v)<_1(prod)n|(raw)(hex){3}(v)<_5(prod)'}


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:

# 6. Training arguments - lower learning rate, gradient clipping, fp16 if CUDA available
training_args = TrainingArguments(
    output_dir="./byt5_finetuned",
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    save_steps=100,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
)

# 7. Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# 8. Train
trainer.train()

# 9. Evaluate
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

<ipython-input-34-1b4beaf53c53>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
50,0.000000,nan
100,0.000000,nan
150,0.000000,nan


Step,Training Loss,Validation Loss
50,0.000000,nan
100,0.000000,nan
150,0.000000,nan
200,0.000000,nan
250,0.000000,nan
300,0.000000,nan
350,0.000000,nan
